<a href="https://colab.research.google.com/github/pablohenrique93/projeto_final_saude_covid19/blob/main/projeto_final_treat_pandas_health_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#INSTALAÇÃO DAS BIBLIOTECAS

In [ ]:
#Instalação do Pandera
!pip install pandera

In [ ]:
#Instalação do PySpark e Gcsfs
!pip install PYSPARK
!pip install gcsfs

In [ ]:
#Instalando bibliotecas SQL
!pip install mysql-connector-python
!pip install pymysql

#IMPORTAÇÃO BIBLIOTECAS

In [ ]:
# Importação das bibliotecas necessárias

import pandas as pd
import pandera as pa
from pyspark.sql import SparkSession
import pyspark.sql.functions
from pyspark.sql.functions import *
import os
import pyspark.sql.functions as F
import numpy as np
from pyspark.sql.window import Window
from google.cloud import storage
from pyspark.sql.types import *
import pymongo
from pymongo import MongoClient
import mysql.connector
import pymysql
from sqlalchemy import create_engine
import google.auth
from google.auth.transport.requests import Request
import sqlalchemy


In [ ]:
# Preparação dos Df's para envio para GCP.

df = pd.read_json('/content/Europa-Covid-Test.json',encoding = 'ISO-8859-1')
df2 = pd.read_csv('/content/WHO-COVID-19-global-data.csv',encoding = 'ISO-8859-1')
df3 = pd.read_csv('/content/WHO-COVID-19-global-table-data.csv',encoding = 'ISO-8859-1')
df4 = pd.read_csv('/content/vaccination-metadata.csv',encoding = 'ISO-8859-1')

#ENVIO DOS DF'S ORIGINAIS PARA O STORAGE

In [ ]:
# Enviar arquivos originais para GCP

df.to_json('gs://projfinal/brutos/Europa-Covid-Test.json',storage_options={'token':'token_gcp'})
df2.to_csv('gs://projfinal/brutos/WHO-COVID-19-global-data.csv',storage_options={'token':'token_gp'})
df3.to_csv('gs://projfinal/brutos/WHO-COVID-19-global-table-data.csv',storage_options={'token':'token_gcp'})
df4.to_csv('gs://projfinal/brutos/vaccination-metadata.csv',storage_options={'token':'token_gcp'})

#EXTRAÇÃO DA BASE DE DADOS (CLOUD STORAGE)

In [ ]:
# #Service acc

# serviceAccount = '/content/bc26-rdg-ed27-7fdc00e9e239.json'
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# #Extração Arquivos GCP/bucket

# client = storage.Client('/content/bc26-rdg-ed27-7fdc00e9e239.json')
# #CRIAR UMA VARIÁVEL PARA RECEBER O NOME DA BUCKET
# bucket = client.get_bucket('projfinal')

# #ESCOLHER O ARQUIVO DENTRO DA BUCKET
# bucket.blob('Europa-Covid-Test')

# #CRIAR UMA VARIÁVEL PARA RECEBER O CAMINHO DO ARQUIVO
# path1 = 'gs://projfinal/brutos/Europa-Covid-Test.json'

In [ ]:
# client = storage.Client('/content/bc26-rdg-ed27-7fdc00e9e239.json')
# #Criar uma variável para receber o nome do bucket
# bucket = client.get_bucket('projfinal')

# #Escolher o arquivo dentro da bucket
# bucket.blob('WHO-COVID-19-global-data')

# #Criar uma variável para receber o nome do arquivo
# path2 = 'gs://projfinal/brutos/WHO-COVID-19-global-data.csv'

In [ ]:
# client = storage.Client('/content/bc26-rdg-ed27-7fdc00e9e239.json')
# #Criar uma variável para receber o nome do bucket
# bucket = client.get_bucket('projfinal')

# #Escolher o arquivo dentro da bucket
# bucket.blob('WHO-COVID-19-global-table-data')

# #Criar uma variável para receber o nome do arquivo
# path3 = 'gs://projfinal/brutos/WHO-COVID-19-global-table-data.csv'

In [ ]:
# client = storage.Client('/content/bc26-rdg-ed27-7fdc00e9e239.json')
# #Criar uma variável para receber o nome do bucket
# bucket = client.get_bucket('projfinal')

# #Escolher o arquivo dentro da bucket
# bucket.blob('vaccination-metadata')

# #Criar uma variável para receber o nome do arquivo
# path4 = 'gs://projfinal/brutos/vaccination-metadata.csv'

In [ ]:
# # Denominação dos Df's
# df_pd_1 = pd.read_json(path1)
# df_pd_2 = pd.read_csv(path2, low_memory=False)
# df_pd_3 = pd.read_csv(path3, low_memory=False)
# df_pd_4 = pd.read_csv(path4, low_memory=False)

In [ ]:
df_pd_1 = pd.read_json('https://storage.googleapis.com/projfinal/brutos/Europa-Covid-Test.json')
df_pd_2 = pd.read_csv('https://storage.googleapis.com/projfinal/brutos/WHO-COVID-19-global-data.csv')
df_pd_3 = pd.read_csv('https://storage.googleapis.com/projfinal/brutos/WHO-COVID-19-global-table-data.csv')
df_pd_4 = pd.read_csv('https://storage.googleapis.com/projfinal/brutos/vaccination-metadata.csv')

In [ ]:
# Verificação se os Dataframes foram devidamente carregados

df_pd_1.head(2)
df_pd_2.head(2)
df_pd_3.head(2)
df_pd_4.head(2)

#ENVIO DOS DF'S ORIGINAIS PARA MySQL

In [ ]:
#Envio dos DF para MYsql


conexao = 'ip_address'
nome_do_banco = 'database_name'
usuario = 'user'
senha = 'password' 
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
             .format(host=conexao, db=nome_do_banco, user= usuario, pw=senha))

df_pd_1.to_sql("Europa-Covid-Test",engine,index=True,index_label='id')
df_pd_2.to_sql("WHO-COVID-19-global-data",engine,index=True,index_label='id')
df_pd_3.to_sql("WHO-COVID-19-global-table-data",engine,index=True,index_label='id')
df_pd_4.to_sql("vaccination-metadata",engine,index=True,index_label='id')

#BACKUP DOS DF'S

In [ ]:
# Faz-se necessário fazer um backup dos Dataframes antes de qualquer manipulação de dados, em caso de um erro 
# nos processos posteriores os dados estão assegurados.
bckup_df_pd_1 = df_pd_1.copy
bckup_df_pd_2 = df_pd_2.copy
bckup_df_pd_3 = df_pd_3.copy
bckup_df_pd_4 = df_pd_4.copy

#PRÉ-ANÁLISE DOS DF'S ORIGINAIS

In [ ]:
# Foi utilizado os seguintes comandos para realizar a Pré análise dos df's originais
bckup_df_pd_2.head()
bckup_df_pd_3.tail()
bckup_df_pd_4.dtypes
bckup_df_pd_2.shape

#TRATAMENTO DE DADOS DF'S (df_pd_2, df_pd_3, df_pd_4)

##TRATAMENTO DO df_pd_2

In [ ]:
# Verificação de cada coluna com o objetivo de encontrar possíveis inconsistências utilizando o comando unique

pd.unique(df_pd_2['Date_reported'])
pd.unique(df_pd_2['Country_code'])
pd.unique(df_pd_2['Country'])
pd.unique(df_pd_2['WHO_region'])
pd.unique(df_pd_2['New_cases'])
pd.unique(df_pd_2['Cumulative_cases'])
pd.unique(df_pd_2['New_deaths'])
pd.unique(df_pd_2['Cumulative_deaths'])

##LIMPEZA DE INCONSISTÊNCIAS ENCONTRADAS df_pd_2

In [ ]:
# Optamos por dropar a coluna 'Unnamed: 0' do df2

df_pd_2.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
# Renomeando as colunas
df_pd_2.rename(columns={
    'Date_reported':'data_relatada',
    'Country_code':'cod_pais',
    'Country':'pais',
    'WHO_region':'regiao_who',
    'New_cases':'novos_casos',
    'Cumulative_cases':'casos_cumulativos',
    'New_deaths':'novas_mortes',
    'Cumulative_deaths':'mortes_cumulativas'
},inplace=True)

In [ ]:
df_pd_2.head(2)

In [ ]:
# Renomeando as colunas e valores da coluna 'pais'

df_pd_2.replace({'pais':'Afghanistan'}, 'Afeganistão',regex=True, inplace=True)  
df_pd_2.replace({'pais':'Albania'}, 'Albânia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Algeria'}, 'Argélia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'American Samoa'}, 'Samoa Americana', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Andorra'}, 'Andorra', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Angola'}, 'Angola', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Anguilla'}, 'Anguilla', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Antigua and Barbuda'}, 'Antígua e Barbuda', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Argentina'}, 'Argentina', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Armenia'}, 'Armênia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Aruba'}, 'Aruba', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Australia'}, 'Austrália', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Austria'}, 'Austria', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Azerbaijan'}, 'Azerbaijão', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Bahamas'}, 'Bahamas', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Bahrain'}, 'Bahrein', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Bangladesh'}, 'Bangladesh', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Barbados'}, 'Barbados', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Belarus'}, 'Bielorrússia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Belgium'}, 'Bélgica', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Belize'}, 'Belize', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Benin'}, 'Benin', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Bermuda'}, 'Bermuda', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Bhutan'}, 'Butão', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Bolivia (Plurinational State of)'}, 'Bolívia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Bonaire'}, 'Bonaire',regex=True, inplace=True)   
df_pd_2.replace({'pais':'Bosnia and Herzegovina'}, 'Bósnia e Herzegovina', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Botswana'}, 'Botsuana', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Brazil'}, 'Brasil', regex=True, inplace=True)  
df_pd_2.replace({'pais':'British Virgin Islands'}, 'Ilhas Virgens Britânicas', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Brunei Darussalam'}, 'Brunei Darussalam', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Bulgaria'}, 'Bulgária', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Burkina Faso'}, 'Burkina Faso', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Burundi'}, 'Burundi', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Cabo Verde'}, 'Cabo Verde', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Cambodia'}, 'Cambodja', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Cameroon'}, 'Camarões', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Canada'}, 'Canadá', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Cayman Islands'}, 'Ilhas Cayman', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Central African Republic'}, 'República Centro-Africana', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Chad'}, 'Chade', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Chile'}, 'Chile', regex=True, inplace=True)  
df_pd_2.replace({'pais':'China'}, 'China', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Colombia'}, 'Colômbia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Comoros'}, 'Comores', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Congo'}, 'Congo', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Cook Islands'}, 'Ilhas Cook', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Costa Rica'}, 'Costa Rica', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Côte d’Ivoire'}, 'Costa do Marfim', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Croatia'}, 'Croácia', regex=True, inplace=True)
df_pd_2.replace({'pais':'Cuba'}, 'Cuba', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Curaçao'}, 'Curaçao', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Cyprus'}, 'Chipre', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Czechia'}, 'Tchéquia', regex=True, inplace=True)  
df_pd_2.replace({'pais':"Democratic People's Republic of Korea"}, 'República Popular Democrática da Coreia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Democratic Republic of the Congo'}, 'República Democrática do Congo', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Denmark'}, 'Dinamarca', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Djibouti'}, 'Djibuti', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Dominica'}, 'Domínica', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Dominican Republic'}, 'República Dominicana', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Ecuador'}, 'Equador', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Egypt'}, 'Egito', regex=True, inplace=True)  
df_pd_2.replace({'pais':'El Salvador'}, 'El Salvador', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Equatorial Guinea'}, 'Guiné Equatorial', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Eritrea'}, 'Eritreia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Estonia'}, 'Estônia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Eswatini'}, 'Essuatíni', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Ethiopia'}, 'Etiópia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Falkland Islands (Malvinas)'}, 'Malvinas', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Faroe Islands'}, 'Ilhas Faroé', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Fiji'}, 'Fiji', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Finland'}, 'Finlândia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'France'}, 'França', regex=True, inplace=True)  
df_pd_2.replace({'pais':'French Guiana'}, 'Guiana Francesa', regex=True, inplace=True)  
df_pd_2.replace({'pais':'French Polynesia'}, 'Polinésia Francesa', regex=True, inplace=True)
df_pd_2.replace({'pais':'Gabon'}, 'Gabão', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Gambia'}, 'Gâmbia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Georgia'}, 'Geórgia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Germany'}, 'Alemanha', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Ghana'}, 'Gana', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Gibraltar'}, 'Gibraltar', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Greece'}, 'Grécia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Greenland'}, 'Groenlândia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Grenada'}, 'Granada', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Guadeloupe'}, 'Guadalupe', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Guam'}, 'Guam', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Guatemala'}, 'Guatemala', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Guernsey'}, 'Guernesey', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Guinea'}, 'Guiné', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Guinea-Bissau'}, 'Guiné-Bissau', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Guyana'}, 'Guiana', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Haiti'}, 'Haiti', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Holy See'}, 'Vaticano', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Honduras'}, 'Honduras', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Hungary'}, 'Hungria', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Iceland'}, 'Islândia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'India'}, 'Índia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Indonesia'}, 'Indonésia',regex=True, inplace=True)   
df_pd_2.replace({'pais':'Iran (Islamic Republic of)'}, 'Irã', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Iraq'}, 'Iraque', regex=True, inplace=True)
df_pd_2.replace({'pais':'Ireland'}, 'Irlanda', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Isle of Man'}, 'Ilha de Man', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Israel'}, 'Israel', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Italy'}, 'Itália', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Jamaica'}, 'Jamaica', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Japan'}, 'Japão', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Jersey'}, 'Jersey', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Jordan'}, 'Jordânia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Kazakhstan'}, 'Cazaquistão', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Kenya'}, 'Quênia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Kiribati'}, 'Kiribati', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Kosovo[1]'}, 'Kosovo', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Kuwait'}, 'Kuwait', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Kyrgyzstan'}, 'Quirguistão', regex=True, inplace=True)  
df_pd_2.replace({'pais':"Lao People's Democratic Republic"}, 'República Democrática Popular do Laos', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Latvia'}, 'Letônia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Lebanon'}, 'Líbano', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Lesotho'}, 'Lesoto', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Liberia'}, 'Libéria', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Libya'}, 'Líbia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Liechtenstein'}, 'Liechtenstein', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Lithuania'}, 'Lituânia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Luxembourg'}, 'Luxemburgo', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Madagascar'}, 'Madagascar', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Malawi'}, 'Malawi', regex=True, inplace=True)
df_pd_2.replace({'pais':'Malaysia'}, 'Malásia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Maldives'}, 'Maldivas', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Mali'}, 'Mali', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Malta'}, 'Malta', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Marshall Islands'}, 'Ilhas Marshall',regex=True, inplace=True)   
df_pd_2.replace({'pais':'Martinique'}, 'Martinica', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Mauritania'}, 'Mauritânia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Mauritius'}, 'Maurício', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Mayotte'}, 'Maiote', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Mexico'}, 'México', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Micronesia (Federated States of)'}, 'Estados Federados da Micronésia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Monaco'}, 'Mônaco', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Mongolia'}, 'Mongólia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Montenegro'}, 'Montenegro', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Montserrat'}, 'Montserrat', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Morocco'}, 'Marrocos', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Mozambique'}, 'Moçambique', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Myanmar'}, 'Mianmar', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Namibia'}, 'Namíbia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Nauru'}, 'Nauru', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Nepal'}, 'Nepal', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Netherlands'}, 'Países Baixos', regex=True, inplace=True)  
df_pd_2.replace({'pais':'New Caledonia'}, 'Nova Caledônia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'New Zealand'}, 'Nova Zelândia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Nicaragua'}, 'Nicarágua', regex=True, inplace=True)
df_pd_2.replace({'pais':'Niger'}, 'Níger', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Nigeria'}, 'Nigéria', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Niue'}, 'Niue', regex=True, inplace=True)  
df_pd_2.replace({'pais':'North Macedonia'}, 'Macedônia do Norte', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Northern Mariana Islands (Commonwealth of the)'}, 'Comunidades das Ilhas Marianas do Norte', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Norway'}, 'Noruega', regex=True, inplace=True)  
df_pd_2.replace({'pais':'occupied Palestinian territory, including east Jerusalem'}, 'Território Palestino ocupado, incluindo Jerusalém Oriental',regex=True, inplace=True)   
df_pd_2.replace({'pais':'Oman'}, 'Omã', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Other'}, 'Other',regex=True, inplace=True)   
df_pd_2.replace({'pais':'Pakistan'}, 'Paquistão', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Palau'}, 'Palau', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Panama'}, 'Panamá', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Papua New Guinea'}, 'Papua Nova Guiné', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Paraguay'}, 'Paraguai', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Peru'}, 'Peru', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Philippines'}, 'Filipinas', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Pitcairn Islands'}, 'Ilhas Pitcairn', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Poland'}, 'Polônia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Portugal'}, 'Portugal', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Puerto Rico'}, 'Porto Rico', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Qatar'}, 'Catar', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Republic of Korea'}, 'República da Coreia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Republic of Moldova'}, 'República da Moldávia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Réunion'}, 'Réunion', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Romania'}, 'Romênia', regex=True, inplace=True)
df_pd_2.replace({'pais':'Russian Federation'}, 'Federação Russa', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Rwanda'}, 'Ruanda', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Saba'}, 'Saba', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Saint Barthélemy'}, 'São Bartolomeu', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Saint Helena, Ascension and Tristan da Cunha'}, 'Santa Helena, Ascensão e Tristão da Cunha', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Saint Kitts and Nevis'}, 'São Cristóvão e Nevis', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Saint Lucia'}, 'Santa Lúcia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Saint Martin'}, 'São Martinho', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Saint Pierre and Miquelon'}, 'São Pedro e Miquelon', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Saint Vincent and the Grenadines'}, 'São Vicente e Granadinas', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Samoa'}, 'Samoa', regex=True, inplace=True)  
df_pd_2.replace({'pais':'San Marino'}, 'San Marino', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Sao Tome and Principe'}, 'São Tomé e Príncipe', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Saudi Arabia'}, 'Arábia Saudita', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Senegal'}, 'Senegal', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Serbia'}, 'Sérvia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Seychelles'}, 'Seicheles',regex=True, inplace=True)   
df_pd_2.replace({'pais':'Sierra Leone'}, 'Serra Leoa', regex=True, inplace=True)        
df_pd_2.replace({'pais':'Singapore'}, 'Cingapura', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Sint Eustatius'}, 'Santo Eustáquio', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Sint Maarten'}, 'Sint Maarten', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Slovakia'}, 'Eslováquia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Slovenia'}, 'Eslovênia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Solomon Islands'}, 'Ilhas Salomão', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Somalia'}, 'Somália', regex=True, inplace=True)
df_pd_2.replace({'pais':'South Africa'}, 'África do Sul', regex=True, inplace=True)  
df_pd_2.replace({'pais':'South Sudan'}, 'Sudão do Sul', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Spain'}, 'Espanha', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Sri Lanka'}, 'Sri Lanka', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Sudan'}, 'Sudão', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Suriname'}, 'Suriname', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Sweden'}, 'Suécia',regex=True, inplace=True)   
df_pd_2.replace({'pais':'Switzerland'}, 'Suiça', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Syrian Arab Republic'}, 'República Árabe da Síria',  regex=True, inplace=True)        
df_pd_2.replace({'pais':'Tajikistan'}, 'Tajiquistão', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Thailand'}, 'Tailândia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'The United Kingdom'}, 'Reino Unido', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Timor-Leste'}, 'Timor-Leste', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Togo'}, 'Togo', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Tokelau'}, 'Tokelau', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Tonga'}, 'Tonga', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Trinidad and Tobago'}, 'Trindade e Tobago',regex=True, inplace=True)   
df_pd_2.replace({'pais':'Tunisia'}, 'Tunísia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Türkiye'}, 'Turquia',regex=True, inplace=True)   
df_pd_2.replace({'pais':'Turkmenistan'}, 'Turquemenistão', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Turks and Caicos Islands'}, 'Ilhas Turcas e Caicos',  regex=True, inplace=True)        
df_pd_2.replace({'pais':'Tuvalu'}, 'Tuvalu', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Uganda'}, 'Uganda',regex=True, inplace=True)  
df_pd_2.replace({'pais':'Ukraine'}, 'Ucrânia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'United Arab Emirates'}, 'Emirados Árabes Unidos',regex=True, inplace=True)
df_pd_2.replace({'pais':'United Republic of Tanzania'}, 'República Unida da Tanzânia', regex=True, inplace=True)  
df_pd_2.replace({'pais':'United States of America'}, 'Estados Unidos da America', regex=True, inplace=True)  
df_pd_2.replace({'pais':'United States Virgin Islands'}, 'Ilhas Virgens dos Estados Unidos', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Uruguay'}, 'Uruguai', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Uzbekistan'}, 'Uzbequistão',regex=True, inplace=True)   
df_pd_2.replace({'pais':'Vanuatu'}, 'Vanuatu',  regex=True, inplace=True)        
df_pd_2.replace({'pais':'Venezuela (Bolivarian Republic of)'}, 'República Bolivariana da Venezuela', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Viet Nam'}, 'Vietnã', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Wallis and Futuna'}, 'Wallis e Futuna',regex=True, inplace=True)  
df_pd_2.replace({'pais':'Yemen'}, 'Iémen', regex=True, inplace=True)  
df_pd_2.replace({'pais':'Zambia'}, 'Zâmbia',regex=True, inplace=True)  
df_pd_2.replace({'pais':'Zimbabwe'}, 'Zimbábue', regex=True, inplace=True)

In [ ]:
df_pd_2.head(2)

In [ ]:
# Renomeando a colunas 'região_Who'

df_pd_2.replace({'regiao_who':'EMRO'}, 'Mediterrâneo Oriental',regex=True, inplace=True)
df_pd_2.replace({'regiao_who':'EURO'}, 'Europa',regex=True, inplace=True)
df_pd_2.replace({'regiao_who':'AFRO'}, 'África',regex=True, inplace=True)
df_pd_2.replace({'regiao_who':'WPRO'}, 'Pacífico Ocidental',regex=True, inplace=True)
df_pd_2.replace({'regiao_who':'AMRO'}, 'Américas',regex=True, inplace=True)
df_pd_2.replace({'regiao_who':'SEARO'}, 'Sudeste Asiático',regex=True, inplace=True)
df_pd_2.replace({'regiao_who':'Other'}, 'Outro',regex=True, inplace=True)

In [ ]:
# Visualizando as alterações feitas
df_pd_2.head(2)
# 257382 rows × 8 columns

In [ ]:
# Verificando a quantidade de valores nulos
df_pd_2.isna().sum()

REALIZANDO OS FILTROS NAS REGIÕES

In [ ]:
# Filtros para retorno dos valores da coluna 'novos_casos', 'casos_cumulativos', 'novas_mortes' e 'mortes_cumulativas' em que somente houveram registros

In [ ]:
# Filtro na região EMRO

In [ ]:
# Filtro parte I
filtro1 = df_pd_2.regiao_who == 'Mediterrâneo Oriental'
df_f1 = df_pd_2.loc[filtro1]
df_f1
# 23892 rows

In [ ]:
# Filtro parte II
filtro1_1 = (df_f1.novos_casos != 0) | (df_f1.casos_cumulativos != 0) | (df_f1.novas_mortes != 0) | (df_f1.mortes_cumulativas != 0)
df_f1_1 = df_f1.loc[filtro1_1]
df_f1_1
# novos casos 18536

In [ ]:
# Filtro na região EURO

In [ ]:
# Filtro parte I
filtro2 = df_pd_2.regiao_who == 'Europa'
df_f2 = df_pd_2.loc[filtro2]
df_f2

In [ ]:
# Filtro parte II - Filtro para remover linhas que sejam todas com valores Nan/0
filtro2_1 = (df_f2.novos_casos != 0) | (df_f2.casos_cumulativos != 0) | (df_f2.novas_mortes != 0) | (df_f2.mortes_cumulativas != 0)
df_f2_1 = df_f2.loc[filtro2_1]
df_f2_1

In [ ]:
# Filtro na região AFRO

In [ ]:
# Filtro parte I
filtro3 = df_pd_2.regiao_who == 'África'
df_f3 = df_pd_2.loc[filtro3]
df_f3

In [ ]:
# Filtro parte II
filtro3_1 = (df_f3.novos_casos != 0) | (df_f3.casos_cumulativos != 0) | (df_f3.novas_mortes != 0) | (df_f3.mortes_cumulativas != 0)
df_f3_1 = df_f3.loc[filtro3_1]
df_f3_1


In [ ]:
# Filtro da região WPRO

In [ ]:
# Filtro parte I
filtro4 = df_pd_2.regiao_who == 'Pacífico Ocidental'
df_f4 = df_pd_2.loc[filtro4]
df_f4

In [ ]:
# Filtro parte II
filtro4_1 = (df_f4.novos_casos != 0) | (df_f4.casos_cumulativos != 0) | (df_f4.novas_mortes != 0) | (df_f4.mortes_cumulativas != 0)
df_f4_1 = df_f4.loc[filtro4_1]
df_f4_1

In [ ]:
# Filtro da região AMRO

In [ ]:
# Filtro parte I
filtro5 = df_pd_2.regiao_who == 'Américas'
df_f5 = df_pd_2.loc[filtro5]
df_f5

In [ ]:
# Filtro parte II
filtro5_1 = (df_f5.novos_casos != 0) | (df_f5.casos_cumulativos != 0) | (df_f5.novas_mortes != 0) | (df_f5.mortes_cumulativas != 0)
df_f5_1 = df_f5.loc[filtro5_1]
df_f5_1

In [ ]:
# Filtro da região SEARO

In [ ]:
# Filtro parte I
filtro6 = df_pd_2.regiao_who == 'Sudeste Asiático'
df_f6 = df_pd_2.loc[filtro6]
df_f6

In [ ]:
# Filtro parte II
filtro6_1 = (df_f6.novos_casos != 0) | (df_f6.casos_cumulativos != 0) | (df_f6.novas_mortes != 0) | (df_f6.mortes_cumulativas != 0)
df_f6_1 = df_f6.loc[filtro6_1]
df_f6_1
 

In [ ]:
# Filtro das regiões não informadas

In [ ]:
# Filtro parte I
filtro7 = df_pd_2.regiao_who == 'Outro'
df_f7 = df_pd_2.loc[filtro7]
df_f7

In [ ]:
# Filtro parte II
filtro7_1 = (df_f7.novos_casos != 0) | (df_f7.casos_cumulativos != 0) | (df_f7.novas_mortes != 0) | (df_f7.mortes_cumulativas != 0)
df_f7_1 = df_f7.loc[filtro7_1]
df_f7_1

CONCATENANDO OS DATAS FRAMES QUE FORAM FILTRADOS EM UM SÓ

In [ ]:
# Foi feito a concatenação 
df_pd_2_final = pd.concat([df_f1_1, df_f2_1, df_f3_1, df_f4_1, df_f5_1, df_f6_1, df_f7_1], ignore_index = True)
# Total de linhas inicial 257382, final 29841 - 161385 

In [ ]:
# Verificação da concatenação
df_pd_2_final.head(3)

##TRATAMENTO DO df_pd_3

In [ ]:
# Verificação de cada coluna com o objetivo de encontrar possíveis inconsistências

In [ ]:
df_pd_3.head(2)

In [ ]:
# Renomear a coluna 'Unnamed: 0' para 'pais'

df_pd_3.rename(columns={'Unnamed: 0':'pais'}, inplace=True)

In [ ]:
# Verificação de cada coluna com o objetivo de encontrar possíveis inconsistências utilizando o comando unique

pd.unique(df_pd_3['pais'])
pd.unique(df_pd_3['Name'])
pd.unique(df_pd_3['WHO Region'])
pd.unique(df_pd_3['Cases - cumulative total'])
pd.unique(df_pd_3['Cases - cumulative total per 100000 population'])
pd.unique(df_pd_3['Cases - newly reported in last 7 days'])
pd.unique(df_pd_3['Cases - newly reported in last 7 days per 100000 population'])
pd.unique(df_pd_3['Cases - newly reported in last 24 hours'])
pd.unique(df_pd_3['Deaths - cumulative total'])
pd.unique(df_pd_3['Deaths - cumulative total per 100000 population'])
pd.unique(df_pd_3['Deaths - newly reported in last 7 days'])
pd.unique(df_pd_3['Deaths - newly reported in last 7 days per 100000 population'])
pd.unique(df_pd_3['Deaths - newly reported in last 24 hours'])

array([nan])

##LIMPEZA DE INCONSISTÊNCIAS ENCONTRADAS df_pd_3

In [ ]:
# Antes da limpeza das colunas, faz-se necessário renomear assim como traduzir, para além de facilitar o entendimento
# dos datasets, facilitar tambem a manipulação dos dados.

In [ ]:
# Renomear colunas da tabela 3

df_pd_3.rename(columns={
                   'Unnamed: 0':'pais',
                   'Name':'regiao',
                   'WHO Region':'oms_regiao2',
                   'Cases - cumulative total':'casos_cumulativos_totais',
                   'Cases - cumulative total per 100000 population':'casos_cumulativos_por_100mil_hab',
                   'Cases - newly reported in last 7 days':'casos_nos_ultimos_7dias',
                   'Cases - newly reported in last 7 days per 100000 population':'casos_nos_ultimos_7dias_por_100mil_hab',
                   'Cases - newly reported in last 24 hours':'casos_nas_ultimas_24hrs',
                   'Deaths - cumulative total':'mortes_cumulativas_totais',
                   'Deaths - cumulative total per 100000 population':'mortes_cumulativas_totais_por_100mil_hab',
                   'Deaths - newly reported in last 7 days':'mortes_nos_ultimos_7dias',
                   'Deaths - newly reported in last 7 days per 100000 population':'mortes_nos_ultimos_7dias_por_100mil_hab',
                   'Deaths - newly reported in last 24 hours':'mortes_nas_ultimas_24hrs',
                   'Unnamed: 0':'pais2'              
                  }, inplace=True)

In [ ]:
df_pd_3.head(2)

In [ ]:
# Renomear nome dos países da tabela 3

df_pd_3.replace({'pais':'Afghanistan'}, 'Afeganistão',regex=True, inplace=True)  
df_pd_3.replace({'pais':'Albania'}, 'Albânia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Algeria'}, 'Argélia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'American Samoa'}, 'Samoa Americana', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Andorra'}, 'Andorra', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Angola'}, 'Angola', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Anguilla'}, 'Anguilla', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Antigua and Barbuda'}, 'Antígua e Barbuda', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Argentina'}, 'Argentina', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Armenia'}, 'Armênia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Aruba'}, 'Aruba', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Australia'}, 'Austrália', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Austria'}, 'Austria', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Azerbaijan'}, 'Azerbaijão', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Bahamas'}, 'Bahamas', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Bahrain'}, 'Bahrein', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Bangladesh'}, 'Bangladesh', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Barbados'}, 'Barbados', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Belarus'}, 'Bielorrússia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Belgium'}, 'Bélgica', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Belize'}, 'Belize', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Benin'}, 'Benin', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Bermuda'}, 'Bermuda', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Bhutan'}, 'Butão', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Bolivia (Plurinational State of)'}, 'Bolívia', regex=True, inplace=True)
df_pd_3.replace({'pais':'Bonaire'}, 'Bonaire',regex=True, inplace=True)   
df_pd_3.replace({'pais':'Bosnia and Herzegovina'}, 'Bósnia e Herzegovina', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Botswana'}, 'Botsuana', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Brazil'}, 'Brasil', regex=True, inplace=True)  
df_pd_3.replace({'pais':'British Virgin Islands'}, 'Ilhas Virgens Britânicas', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Brunei Darussalam'}, 'Brunei Darussalam', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Bulgaria'}, 'Bulgária', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Burkina Faso'}, 'Burkina Faso', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Burundi'}, 'Burundi', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Cabo Verde'}, 'Cabo Verde', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Cambodia'}, 'Cambodja', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Cameroon'}, 'Camarões', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Canada'}, 'Canadá', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Cayman Islands'}, 'Ilhas Cayman', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Central African Republic'}, 'República Centro-Africana', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Chad'}, 'Chade', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Chile'}, 'Chile', regex=True, inplace=True)  
df_pd_3.replace({'pais':'China'}, 'China', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Colombia'}, 'Colômbia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Comoros'}, 'Comores', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Congo'}, 'Congo', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Cook Islands'}, 'Ilhas Cook', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Costa Rica'}, 'Costa Rica', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Côte d’Ivoire'}, 'Costa do Marfim', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Croatia'}, 'Croácia', regex=True, inplace=True)
df_pd_3.replace({'pais':'Cuba'}, 'Cuba', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Curaçao'}, 'Curaçao', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Cyprus'}, 'Chipre', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Czechia'}, 'Tchéquia', regex=True, inplace=True)  
df_pd_3.replace({'pais':"Democratic People's Republic of Korea"}, 'República Popular Democrática da Coreia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Democratic Republic of the Congo'}, 'República Democrática do Congo', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Denmark'}, 'Dinamarca', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Djibouti'}, 'Djibuti', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Dominica'}, 'Domínica', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Dominican Republic'}, 'República Dominicana', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Ecuador'}, 'Equador', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Egypt'}, 'Egito', regex=True, inplace=True)  
df_pd_3.replace({'pais':'El Salvador'}, 'El Salvador', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Equatorial Guinea'}, 'Guiné Equatorial', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Eritrea'}, 'Eritreia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Estonia'}, 'Estônia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Eswatini'}, 'Essuatíni', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Ethiopia'}, 'Etiópia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Falkland Islands (Malvinas)'}, 'Malvinas', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Faroe Islands'}, 'Ilhas Faroé', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Fiji'}, 'Fiji', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Finland'}, 'Finlândia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'France'}, 'França', regex=True, inplace=True)  
df_pd_3.replace({'pais':'French Guiana'}, 'Guiana Francesa', regex=True, inplace=True)  
df_pd_3.replace({'pais':'French Polynesia'}, 'Polinésia Francesa', regex=True, inplace=True)
df_pd_3.replace({'pais':'Gabon'}, 'Gabão', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Gambia'}, 'Gâmbia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Georgia'}, 'Geórgia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Germany'}, 'Alemanha', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Ghana'}, 'Gana', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Gibraltar'}, 'Gibraltar', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Greece'}, 'Grécia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Greenland'}, 'Groenlândia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Grenada'}, 'Granada', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Guadeloupe'}, 'Guadalupe', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Guam'}, 'Guam', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Guatemala'}, 'Guatemala', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Guernsey'}, 'Guernesey', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Guinea'}, 'Guiné', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Guinea-Bissau'}, 'Guiné-Bissau', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Guyana'}, 'Guiana', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Haiti'}, 'Haiti', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Holy See'}, 'Vaticano', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Honduras'}, 'Honduras', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Hungary'}, 'Hungria', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Iceland'}, 'Islândia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'India'}, 'Índia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Indonesia'}, 'Indonésia',regex=True, inplace=True)   
df_pd_3.replace({'pais':'Iran (Islamic Republic of)'}, 'Irã', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Iraq'}, 'Iraque', regex=True, inplace=True)
df_pd_3.replace({'pais':'Ireland'}, 'Irlanda', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Isle of Man'}, 'Ilha de Man', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Israel'}, 'Israel', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Italy'}, 'Itália', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Jamaica'}, 'Jamaica', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Japan'}, 'Japão', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Jersey'}, 'Jersey', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Jordan'}, 'Jordânia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Kazakhstan'}, 'Cazaquistão', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Kenya'}, 'Quênia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Kiribati'}, 'Kiribati', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Kosovo[1]'}, 'Kosovo', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Kuwait'}, 'Kuwait', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Kyrgyzstan'}, 'Quirguistão', regex=True, inplace=True)  
df_pd_3.replace({'pais':"Lao People's Democratic Republic"}, 'República Democrática Popular do Laos', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Latvia'}, 'Letônia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Lebanon'}, 'Líbano', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Lesotho'}, 'Lesoto', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Liberia'}, 'Libéria', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Libya'}, 'Líbia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Liechtenstein'}, 'Liechtenstein', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Lithuania'}, 'Lituânia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Luxembourg'}, 'Luxemburgo', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Madagascar'}, 'Madagascar', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Malawi'}, 'Malawi', regex=True, inplace=True)
df_pd_3.replace({'pais':'Malaysia'}, 'Malásia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Maldives'}, 'Maldivas', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Mali'}, 'Mali', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Malta'}, 'Malta', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Marshall Islands'}, 'Ilhas Marshall',regex=True, inplace=True)   
df_pd_3.replace({'pais':'Martinique'}, 'Martinica', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Mauritania'}, 'Mauritânia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Mauritius'}, 'Maurício', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Mayotte'}, 'Maiote', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Mexico'}, 'México', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Micronesia (Federated States of)'}, 'Estados Federados da Micronésia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Monaco'}, 'Mônaco', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Mongolia'}, 'Mongólia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Montenegro'}, 'Montenegro', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Montserrat'}, 'Montserrat', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Morocco'}, 'Marrocos', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Mozambique'}, 'Moçambique', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Myanmar'}, 'Mianmar', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Namibia'}, 'Namíbia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Nauru'}, 'Nauru', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Nepal'}, 'Nepal', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Netherlands'}, 'Países Baixos', regex=True, inplace=True)  
df_pd_3.replace({'pais':'New Caledonia'}, 'Nova Caledônia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'New Zealand'}, 'Nova Zelândia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Nicaragua'}, 'Nicarágua', regex=True, inplace=True)
df_pd_3.replace({'pais':'Niger'}, 'Níger', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Nigeria'}, 'Nigéria', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Niue'}, 'Niue', regex=True, inplace=True)  
df_pd_3.replace({'pais':'North Macedonia'}, 'Macedônia do Norte', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Northern Mariana Islands (Commonwealth of the)'}, 'Comunidades das Ilhas Marianas do Norte', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Norway'}, 'Noruega', regex=True, inplace=True)  
df_pd_3.replace({'pais':'occupied Palestinian territory, including east Jerusalem'}, 'Território Palestino ocupado, incluindo Jerusalém Oriental',regex=True, inplace=True)   
df_pd_3.replace({'pais':'Oman'}, 'Omã', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Other'}, 'Other',regex=True, inplace=True)   
df_pd_3.replace({'pais':'Pakistan'}, 'Paquistão', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Palau'}, 'Palau', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Panama'}, 'Panamá', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Papua New Guinea'}, 'Papua Nova Guiné', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Paraguay'}, 'Paraguai', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Peru'}, 'Peru', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Philippines'}, 'Filipinas', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Pitcairn Islands'}, 'Ilhas Pitcairn', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Poland'}, 'Polônia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Portugal'}, 'Portugal', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Puerto Rico'}, 'Porto Rico', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Qatar'}, 'Catar', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Republic of Korea'}, 'República da Coreia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Republic of Moldova'}, 'República da Moldávia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Réunion'}, 'Réunion', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Romania'}, 'Romênia', regex=True, inplace=True)
df_pd_3.replace({'pais':'Russian Federation'}, 'Federação Russa', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Rwanda'}, 'Ruanda', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Saba'}, 'Saba', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Saint Barthélemy'}, 'São Bartolomeu', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Saint Helena, Ascension and Tristan da Cunha'}, 'Santa Helena, Ascensão e Tristão da Cunha', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Saint Kitts and Nevis'}, 'São Cristóvão e Nevis', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Saint Lucia'}, 'Santa Lúcia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Saint Martin'}, 'São Martinho', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Saint Pierre and Miquelon'}, 'São Pedro e Miquelon', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Saint Vincent and the Grenadines'}, 'São Vicente e Granadinas', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Samoa'}, 'Samoa', regex=True, inplace=True)  
df_pd_3.replace({'pais':'San Marino'}, 'San Marino', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Sao Tome and Principe'}, 'São Tomé e Príncipe', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Saudi Arabia'}, 'Arábia Saudita', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Senegal'}, 'Senegal', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Serbia'}, 'Sérvia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Seychelles'}, 'Seicheles',regex=True, inplace=True)   
df_pd_3.replace({'pais':'Sierra Leone'}, 'Serra Leoa', regex=True, inplace=True)        
df_pd_3.replace({'pais':'Singapore'}, 'Cingapura', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Sint Eustatius'}, 'Santo Eustáquio', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Sint Maarten'}, 'Sint Maarten', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Slovakia'}, 'Eslováquia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Slovenia'}, 'Eslovênia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Solomon Islands'}, 'Ilhas Salomão', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Somalia'}, 'Somália', regex=True, inplace=True)
df_pd_3.replace({'pais':'South Africa'}, 'África do Sul', regex=True, inplace=True)  
df_pd_3.replace({'pais':'South Sudan'}, 'Sudão do Sul', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Spain'}, 'Espanha', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Sri Lanka'}, 'Sri Lanka', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Sudan'}, 'Sudão', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Suriname'}, 'Suriname', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Sweden'}, 'Suécia',regex=True, inplace=True)   
df_pd_3.replace({'pais':'Switzerland'}, 'Suiça', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Syrian Arab Republic'}, 'República Árabe da Síria',  regex=True, inplace=True)        
df_pd_3.replace({'pais':'Tajikistan'}, 'Tajiquistão', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Thailand'}, 'Tailândia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'The United Kingdom'}, 'Reino Unido', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Timor-Leste'}, 'Timor-Leste', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Togo'}, 'Togo', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Tokelau'}, 'Tokelau', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Tonga'}, 'Tonga', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Trinidad and Tobago'}, 'Trindade e Tobago',regex=True, inplace=True)   
df_pd_3.replace({'pais':'Tunisia'}, 'Tunísia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Türkiye'}, 'Turquia',regex=True, inplace=True)   
df_pd_3.replace({'pais':'Turkmenistan'}, 'Turquemenistão', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Turks and Caicos Islands'}, 'Ilhas Turcas e Caicos',  regex=True, inplace=True)        
df_pd_3.replace({'pais':'Tuvalu'}, 'Tuvalu', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Uganda'}, 'Uganda',regex=True, inplace=True)  
df_pd_3.replace({'pais':'Ukraine'}, 'Ucrânia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'United Arab Emirates'}, 'Emirados Árabes Unidos',regex=True, inplace=True)
df_pd_3.replace({'pais':'United Republic of Tanzania'}, 'República Unida da Tanzânia', regex=True, inplace=True)  
df_pd_3.replace({'pais':'United States of America'}, 'Estados Unidos da America', regex=True, inplace=True)  
df_pd_3.replace({'pais':'United States Virgin Islands'}, 'Ilhas Virgens dos Estados Unidos', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Uruguay'}, 'Uruguai', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Uzbekistan'}, 'Uzbequistão',regex=True, inplace=True)   
df_pd_3.replace({'pais':'Vanuatu'}, 'Vanuatu',  regex=True, inplace=True)        
df_pd_3.replace({'pais':'Venezuela (Bolivarian Republic of)'}, 'República Bolivariana da Venezuela', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Viet Nam'}, 'Vietnã', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Wallis and Futuna'}, 'Wallis e Futuna',regex=True, inplace=True)  
df_pd_3.replace({'pais':'Yemen'}, 'Iémen', regex=True, inplace=True)  
df_pd_3.replace({'pais':'Zambia'}, 'Zâmbia',regex=True, inplace=True)  
df_pd_3.replace({'pais':'Zimbabwe'}, 'Zimbábue', regex=True, inplace=True)

In [ ]:
df_pd_3.head(2)

In [ ]:
# Optamos por dropar a coluna 'oms_regiao2' que descreve o código de cada país 
# pois já existe uma coluna no Dataframe que descreve o país correspondente ao código

df_pd_3.drop('oms_regiao2',axis=1,inplace=True)

In [ ]:
df_pd_3.head(3)

In [ ]:
# Renomeando a coluna região com a respectiva tradução

df_pd_3.replace({'regiao':'Americas'}, 'Americas',regex=True, inplace=True)  
df_pd_3.replace({'regiao':'South-East Asia'}, 'Sudeste da Ásia',regex=True, inplace=True)  
df_pd_3.replace({'regiao':'Europe'}, 'Europa',regex=True, inplace=True)  
df_pd_3.replace({'regiao':'Western Pacific'}, 'Pacífico Ocidental',regex=True, inplace=True)  
df_pd_3.replace({'regiao':'Eastern Mediterranean'}, 'Mediterrâneo oriental',regex=True, inplace=True)
df_pd_3.replace({'regiao':'Africa'}, 'Africa',regex=True, inplace=True)
df_pd_3.replace({'regiao':'Other'}, 'Outro',regex=True, inplace=True)  

In [ ]:
# Verificação das modificações realizadas
df_pd_3.head(3)

##TRATAMENTO DO df_pd_4

In [ ]:
# Visualização do data frame para uma possivel pré análise
df_pd_4.dtypes

In [ ]:
# Verificando os valores nulos
df_pd_4.isna().sum()

In [ ]:
# Verificando o conteudo das colunas atraves do pd.unique
pd.unique(df_pd_4['COMMENT'])

In [ ]:
# Realizando drop de colunas devido as colunas "END_DATE" e "COMMENT", terem apenas valores nulos
df_pd_4.drop(columns=['END_DATE'], inplace = True)
df_pd_4.drop(columns=['COMMENT'], inplace = True)

In [ ]:
# Renomeando as colunas 
# Optamos por renomear a coluna "iso3" para país assim como transformar seus códigos em países correspondentes.
df_pd_4.rename(columns={
    'ISO3':'pais',
    'VACCINE_NAME':'vacina',
    'PRODUCT_NAME':'produto',
    'COMPANY_NAME':'empresa',
    'AUTHORIZATION_DATE':'data_de_autorizacao',
    'START_DATE':'data_de_inicio',
    'DATA_SOURCE':'fonte_de_dados'
},inplace=True)

In [ ]:
# Verificação das alterações realizadas
df_pd_4.head(2)

In [ ]:
# Renomeando as siglas da coluna pais 
df_pd_4.replace({'pais':'SHN'}, 'Santa Helena, Ascensão e Tristão da Cunha', regex=True, inplace=True)
df_pd_4.replace({'pais':'GRL'}, 'Groelândia', regex=True, inplace=True)
df_pd_4.replace({'pais':'FRO'}, 'Ilhas Faroé', regex=True, inplace=True)
df_pd_4.replace({'pais':'BIH'}, 'Bósnia e Herzegovina', regex=True, inplace=True)
df_pd_4.replace({'pais':'JEY'}, 'Jersey', regex=True, inplace=True)
df_pd_4.replace({'pais':'GGY'}, 'Guernsey', regex=True, inplace=True)
df_pd_4.replace({'pais':'GIB'}, 'Gibraltar', regex=True, inplace=True)
df_pd_4.replace({'pais':'IMN'}, 'Ilha de Man', regex=True, inplace=True)
df_pd_4.replace({'pais':'FLK'}, 'Malvinas', regex=True, inplace=True)
df_pd_4.replace({'pais':'RUS'}, 'Federação Russa', regex=True, inplace=True)
df_pd_4.replace({'pais':'TUR'}, 'Turquia', regex=True, inplace=True)
df_pd_4.replace({'pais':'UZB'}, 'Uzbequistão', regex=True, inplace=True)
df_pd_4.replace({'pais':'CHN'}, 'China', regex=True, inplace=True)
df_pd_4.replace({'pais':'TKM'}, 'Turquemenistão', regex=True, inplace=True)
df_pd_4.replace({'pais':'AUT'}, 'Austria', regex=True, inplace=True)
df_pd_4.replace({'pais':'DEU'}, 'Alemanha', regex=True, inplace=True)
df_pd_4.replace({'pais':'VEN'}, 'República Bolivariana da Venezuela', regex=True, inplace=True)
df_pd_4.replace({'pais':'CUB'}, 'Cuba', regex=True, inplace=True)
df_pd_4.replace({'pais':'IRN'}, 'Irã', regex=True, inplace=True)
df_pd_4.replace({'pais':'NIC'}, 'Nicarágua', regex=True, inplace=True)
df_pd_4.replace({'pais':'AZE'}, 'Azerbaijão', regex=True, inplace=True)
df_pd_4.replace({'pais':'KGZ'}, 'Quirguistão', regex=True, inplace=True)
df_pd_4.replace({'pais':'SVK'}, 'Eslováquia', regex=True, inplace=True)
df_pd_4.replace({'pais':'ARM'}, 'Armênia', regex=True, inplace=True)
df_pd_4.replace({'pais':'OMN'}, 'Omã', regex=True, inplace=True)
df_pd_4.replace({'pais':'MDV'}, 'Maldivas', regex=True, inplace=True)
df_pd_4.replace({'pais':'ATG'}, 'Antígua e Barbuda', regex=True, inplace=True)
df_pd_4.replace({'pais':'TJK'}, 'Tajiquistão', regex=True, inplace=True)
df_pd_4.replace({'pais':'LAO'}, 'República Democrática Popular do Laos', regex=True, inplace=True)
df_pd_4.replace({'pais':'MNG'}, 'Mongólia', regex=True, inplace=True)
df_pd_4.replace({'pais':'PHL'}, 'Filipinas', regex=True, inplace=True)
df_pd_4.replace({'pais':'HUN'}, 'Hungria', regex=True, inplace=True)
df_pd_4.replace({'pais':'BLR'}, 'Bielorrússia', regex=True, inplace=True)
df_pd_4.replace({'pais':'PRY'}, 'Paraguai', regex=True, inplace=True)
df_pd_4.replace({'pais':'ARG'}, 'Argentina', regex=True, inplace=True)
df_pd_4.replace({'pais':'BOL'}, 'Bolívia', regex=True, inplace=True)
df_pd_4.replace({'pais':'PSE'}, 'Palestina', regex=True, inplace=True)
df_pd_4.replace({'pais':'TUN'}, 'Tunísia', regex=True, inplace=True)
df_pd_4.replace({'pais':'PAK'}, 'Paquistão', regex=True, inplace=True)
df_pd_4.replace({'pais':'SMR'}, 'San Marino', regex=True, inplace=True)
df_pd_4.replace({'pais':'MNE'}, 'Montenegro', regex=True, inplace=True)
df_pd_4.replace({'pais':'GAB'}, 'Gabão', regex=True, inplace=True)
df_pd_4.replace({'pais':'RWA'}, 'Ruanda', regex=True, inplace=True)
df_pd_4.replace({'pais':'SYC'}, 'Seicheles', regex=True, inplace=True)
df_pd_4.replace({'pais':'ZWE'}, 'Zimbábue', regex=True, inplace=True)
df_pd_4.replace({'pais':'IRQ'}, 'Iraque', regex=True, inplace=True)
df_pd_4.replace({'pais':'MEX'}, 'México', regex=True, inplace=True)
df_pd_4.replace({'pais':'JOR'}, 'Jordânia', regex=True, inplace=True)
df_pd_4.replace({'pais':'DJI'}, 'Djibouti', regex=True, inplace=True)
df_pd_4.replace({'pais':'LBN'}, 'Líbano', regex=True, inplace=True)
df_pd_4.replace({'pais':'KAZ'}, 'Cazaquistão', regex=True, inplace=True)
df_pd_4.replace({'pais':'MKD'}, 'Macedônia', regex=True, inplace=True)
df_pd_4.replace({'pais':'VCT'}, 'São Vicente e Granadinas', regex=True, inplace=True)
df_pd_4.replace({'pais':'DZA'}, 'Argélia', regex=True, inplace=True)
df_pd_4.replace({'pais':'AFG'}, 'Afeganistão', regex=True, inplace=True)
df_pd_4.replace({'pais':'SDN'}, 'Sudão', regex=True, inplace=True)
df_pd_4.replace({'pais':'YEM'}, 'Iêmen', regex=True, inplace=True)
df_pd_4.replace({'pais':'ARE'}, 'Emirados Árabes Unidos', regex=True, inplace=True)
df_pd_4.replace({'pais':'EGY'}, 'Egito', regex=True, inplace=True)
df_pd_4.replace({'pais':'BHR'}, 'Bahrein', regex=True, inplace=True)
df_pd_4.replace({'pais':'MAR'}, 'Camarões', regex=True, inplace=True)
df_pd_4.replace({'pais':'LKA'}, 'Sri Lanka', regex=True, inplace=True)
df_pd_4.replace({'pais':'GUY'}, 'Guiana', regex=True, inplace=True)
df_pd_4.replace({'pais':'KWT'}, 'Kuwait', regex=True, inplace=True)
df_pd_4.replace({'pais':'QAT'}, 'Catar', regex=True, inplace=True)
df_pd_4.replace({'pais':'SAU'}, 'Arábia Saudita', regex=True, inplace=True)
df_pd_4.replace({'pais':'SOM'}, 'Somália', regex=True, inplace=True)
df_pd_4.replace({'pais':'SRB'}, 'Sérvia', regex=True, inplace=True)
df_pd_4.replace({'pais':'COG'}, 'República do Congo', regex=True, inplace=True)
df_pd_4.replace({'pais':'GIN'}, 'Guiné', regex=True, inplace=True)
df_pd_4.replace({'pais':'VNM'}, 'Vietnã', regex=True, inplace=True)
df_pd_4.replace({'pais':'MYS'}, 'Malásia', regex=True, inplace=True)
df_pd_4.replace({'pais':'GHA'}, 'Gana', regex=True, inplace=True)
df_pd_4.replace({'pais':'CIV'}, 'Costa do Marfim', regex=True, inplace=True)
df_pd_4.replace({'pais':'SYR'}, 'Síria', regex=True, inplace=True)
df_pd_4.replace({'pais':'HND'}, 'Honduras', regex=True, inplace=True)
df_pd_4.replace({'pais':'IND'}, 'Índia', regex=True, inplace=True)
df_pd_4.replace({'pais':'NPL'}, 'Nepal', regex=True, inplace=True)
df_pd_4.replace({'pais':'LBY'}, 'Líbia', regex=True, inplace=True)
df_pd_4.replace({'pais':'BGD'}, 'Bangladesh', regex=True, inplace=True)
df_pd_4.replace({'pais':'MDA'}, 'Moldávia', regex=True, inplace=True)
df_pd_4.replace({'pais':'ALB'}, 'Albânia', regex=True, inplace=True)
df_pd_4.replace({'pais':'GTM'}, 'Guatemala', regex=True, inplace=True)
df_pd_4.replace({'pais':'TCD'}, 'Chade', regex=True, inplace=True)
df_pd_4.replace({'pais':'VUT'}, 'Vanuatu', regex=True, inplace=True)
df_pd_4.replace({'pais':'MMR'}, 'Mianmar', regex=True, inplace=True)
df_pd_4.replace({'pais':'KIR'}, 'Quiribati', regex=True, inplace=True)
df_pd_4.replace({'pais':'BDI'}, 'Burundi', regex=True, inplace=True)
df_pd_4.replace({'pais':'BTN'}, 'Butão', regex=True, inplace=True)
df_pd_4.replace({'pais':'LSO'}, 'Lesoto', regex=True, inplace=True)
df_pd_4.replace({'pais':'MDG'}, 'Madagascar', regex=True, inplace=True)
df_pd_4.replace({'pais':'TZA'}, 'Tanzânia', regex=True, inplace=True)
df_pd_4.replace({'pais':'ZMB'}, 'Zâmbia', regex=True, inplace=True)
df_pd_4.replace({'pais':'SGP'}, 'Singapura', regex=True, inplace=True)
df_pd_4.replace({'pais':'SLV'}, 'El Salvador', regex=True, inplace=True)
df_pd_4.replace({'pais':'BFA'}, 'Burkina Faso', regex=True, inplace=True)
df_pd_4.replace({'pais':'CPV'}, 'Cabo Verde', regex=True, inplace=True)
df_pd_4.replace({'pais':'KEN'}, 'Quênia', regex=True, inplace=True)
df_pd_4.replace({'pais':'GMB'}, 'Gâmbia', regex=True, inplace=True)
df_pd_4.replace({'pais':'KHM'}, 'Camboja', regex=True, inplace=True)
df_pd_4.replace({'pais':'PER'}, 'Peru', regex=True, inplace=True)
df_pd_4.replace({'pais':'GNQ'}, 'Guiné Equatorial', regex=True, inplace=True)
df_pd_4.replace({'pais':'JAM'}, 'Jamaica', regex=True, inplace=True)
df_pd_4.replace({'pais':'PRT'}, 'Portugal', regex=True, inplace=True)
df_pd_4.replace({'pais':'DMA'}, 'Dominica', regex=True, inplace=True)
df_pd_4.replace({'pais':'MOZ'}, 'Moçambique', regex=True, inplace=True)
df_pd_4.replace({'pais':'SLB'}, 'Ilhas Salomão', regex=True, inplace=True)
df_pd_4.replace({'pais':'MUS'}, 'Ilhas Maurício', regex=True, inplace=True)
df_pd_4.replace({'pais':'PNG'}, 'Papua-Nova Guiné', regex=True, inplace=True)
df_pd_4.replace({'pais':'BRN'}, 'Brunei', regex=True, inplace=True)
df_pd_4.replace({'pais':'MRT'}, 'Mauritânia', regex=True, inplace=True)
df_pd_4.replace({'pais':'NAM'}, 'Namíbia', regex=True, inplace=True)
df_pd_4.replace({'pais':'COM'}, 'Comores', regex=True, inplace=True)
df_pd_4.replace({'pais':'NER'}, 'Níger', regex=True, inplace=True)
df_pd_4.replace({'pais':'ETH'}, 'Etiópia', regex=True, inplace=True)
df_pd_4.replace({'pais':'IDN'}, 'Indonésia', regex=True, inplace=True)
df_pd_4.replace({'pais':'GNB'}, 'Guiné-Bissau', regex=True, inplace=True)
df_pd_4.replace({'pais':'SLE'}, 'Serra Leoa', regex=True, inplace=True)
df_pd_4.replace({'pais':'SEN'}, 'Senegal', regex=True, inplace=True)
df_pd_4.replace({'pais':'CMR'}, 'Camarões', regex=True, inplace=True)
df_pd_4.replace({'pais':'GEO'}, 'Geórgia', regex=True, inplace=True)
df_pd_4.replace({'pais':'THA'}, 'Tailândia', regex=True, inplace=True)
df_pd_4.replace({'pais':'BRB'}, 'Barbados', regex=True, inplace=True)
df_pd_4.replace({'pais':'SUR'}, 'Suriname', regex=True, inplace=True)
df_pd_4.replace({'pais':'TTO'}, 'Trindade e Tobago', regex=True, inplace=True)
df_pd_4.replace({'pais':'BLZ'}, 'Belize', regex=True, inplace=True)
df_pd_4.replace({'pais':'PCN'}, 'Ilhas Pitcairn', regex=True, inplace=True)
df_pd_4.replace({'pais':'UKR'}, 'Ucrânia', regex=True, inplace=True)
df_pd_4.replace({'pais':'CYP'}, 'Chipre', regex=True, inplace=True)
df_pd_4.replace({'pais':'BEL'}, 'Bélgica', regex=True, inplace=True)
df_pd_4.replace({'pais':'NLD'}, 'Países Baixos', regex=True, inplace=True)
df_pd_4.replace({'pais':'POL'}, 'Polônia', regex=True, inplace=True)
df_pd_4.replace({'pais':'CZE'}, 'Tchéquia', regex=True, inplace=True)
df_pd_4.replace({'pais':'SWE'}, 'Suécia', regex=True, inplace=True)
df_pd_4.replace({'pais':'ROU'}, 'Romênia', regex=True, inplace=True)
df_pd_4.replace({'pais':'LTU'}, 'Lituânia', regex=True, inplace=True)
df_pd_4.replace({'pais':'ESP'}, 'Espanha', regex=True, inplace=True)
df_pd_4.replace({'pais':'LVA'}, 'Letônia', regex=True, inplace=True)
df_pd_4.replace({'pais':'ITA'}, 'Itália', regex=True, inplace=True)
df_pd_4.replace({'pais':'SVN'}, 'Eslovênia', regex=True, inplace=True)
df_pd_4.replace({'pais':'FRA'}, 'França', regex=True, inplace=True)
df_pd_4.replace({'pais':'ISL'}, 'Islândia', regex=True, inplace=True)
df_pd_4.replace({'pais':'BGR'}, 'Bulgária', regex=True, inplace=True)
df_pd_4.replace({'pais':'LUX'}, 'Luxemburgo', regex=True, inplace=True)
df_pd_4.replace({'pais':'IRL'}, 'Irlanda', regex=True, inplace=True)
df_pd_4.replace({'pais':'EST'}, 'Estônia', regex=True, inplace=True)
df_pd_4.replace({'pais':'HRV'}, 'Croácia', regex=True, inplace=True)
df_pd_4.replace({'pais':'FIN'}, 'Finlândia', regex=True, inplace=True)
df_pd_4.replace({'pais':'DNK'}, 'Dinamarca', regex=True, inplace=True)
df_pd_4.replace({'pais':'GRC'}, 'Grécia', regex=True, inplace=True)
df_pd_4.replace({'pais':'VGB'}, 'Ilhas Virgens Britânicas', regex=True, inplace=True)
df_pd_4.replace({'pais':'MSR'}, 'Montserrat', regex=True, inplace=True)
df_pd_4.replace({'pais':'AIA'}, 'Anguilla', regex=True, inplace=True)
df_pd_4.replace({'pais':'MLT'}, 'Malta', regex=True, inplace=True)
df_pd_4.replace({'pais':'CAN'}, 'Canadá', regex=True, inplace=True)
df_pd_4.replace({'pais':'BEN'}, 'Benin', regex=True, inplace=True)
df_pd_4.replace({'pais':'ISR'}, 'Israel', regex=True, inplace=True)
df_pd_4.replace({'pais':'FJI'}, 'Fiji', regex=True, inplace=True)
df_pd_4.replace({'pais':'JPN'}, 'Japão', regex=True, inplace=True)
df_pd_4.replace({'pais':'GBR'}, 'Reino Unido', regex=True, inplace=True)
df_pd_4.replace({'pais':'MCO'}, 'Mônaco', regex=True, inplace=True)
df_pd_4.replace({'pais':'XKX'}, 'Kosovo', regex=True, inplace=True)
df_pd_4.replace({'pais':'STP'}, 'São Tomé e Príncipe', regex=True, inplace=True)
df_pd_4.replace({'pais':'SXM'}, 'Sint Maarten', regex=True, inplace=True)
df_pd_4.replace({'pais':'URY'}, 'Uruguai', regex=True, inplace=True)
df_pd_4.replace({'pais':'ECU'}, 'Equador', regex=True, inplace=True)
df_pd_4.replace({'pais':'BMU'}, 'Bermudas', regex=True, inplace=True)
df_pd_4.replace({'pais':'CUW'}, 'Curaçao', regex=True, inplace=True)
df_pd_4.replace({'pais':'BHS'}, 'Bahamas', regex=True, inplace=True)
df_pd_4.replace({'pais':'KNA'}, 'São Cristóvão e Névis', regex=True, inplace=True)
df_pd_4.replace({'pais':'LCA'}, 'Santa Lúcia', regex=True, inplace=True)
df_pd_4.replace({'pais':'DOM'}, 'República Dominicana', regex=True, inplace=True)
df_pd_4.replace({'pais':'NRU'}, 'Nauru', regex=True, inplace=True)
df_pd_4.replace({'pais':'NOR'}, 'Noruega', regex=True, inplace=True)
df_pd_4.replace({'pais':'COL'}, 'Colômbia', regex=True, inplace=True)
df_pd_4.replace({'pais':'WSM'}, 'Samoa', regex=True, inplace=True)
df_pd_4.replace({'pais':'NZL'}, 'Nova Zelândia', regex=True, inplace=True)
df_pd_4.replace({'pais':'TON'}, 'Tonga', regex=True, inplace=True)
df_pd_4.replace({'pais':'BRA'}, 'Brasil', regex=True, inplace=True)
df_pd_4.replace({'pais':'AUS'}, 'Austrália', regex=True, inplace=True)
df_pd_4.replace({'pais':'KOR'}, 'República da Coreia', regex=True, inplace=True)
df_pd_4.replace({'pais':'FSM'}, 'Estados Federados da Micronésia', regex=True, inplace=True)
df_pd_4.replace({'pais':'TUV'}, 'Tuvalu', regex=True, inplace=True)
df_pd_4.replace({'pais':'CRI'}, 'Costa Rica', regex=True, inplace=True)
df_pd_4.replace({'pais':'AND'}, 'Andorra', regex=True, inplace=True)
df_pd_4.replace({'pais':'GLP'}, 'Guadalupe', regex=True, inplace=True)
df_pd_4.replace({'pais':'TLS'}, 'Timor-Leste', regex=True, inplace=True)
df_pd_4.replace({'pais':'PAN'}, 'Panamá', regex=True, inplace=True)
df_pd_4.replace({'pais':'GRD'}, 'Granada', regex=True, inplace=True)
df_pd_4.replace({'pais':'CHL'}, 'Chile', regex=True, inplace=True)
df_pd_4.replace({'pais':'CHE'}, 'Suíça', regex=True, inplace=True)
df_pd_4.replace({'pais':'MTQ'}, 'Martinica', regex=True, inplace=True)
df_pd_4.replace({'pais':'BWA'}, 'Botsuana', regex=True, inplace=True)
df_pd_4.replace({'pais':'PLW'}, 'Palau', regex=True, inplace=True)
df_pd_4.replace({'pais':'CYM'}, 'Ilhas Cayman', regex=True, inplace=True)
df_pd_4.replace({'pais':'ABW'}, 'Aruba', regex=True, inplace=True)
df_pd_4.replace({'pais':'USA'}, 'Estados Unidos da America', regex=True, inplace=True)
df_pd_4.replace({'pais':'TCA'}, 'Ilhas Turcas e Caicos', regex=True, inplace=True)
df_pd_4.replace({'pais':'SWZ'}, 'Essuatíni', regex=True, inplace=True)
df_pd_4.replace({'pais':'UGA'}, 'Uganda', regex=True, inplace=True)
df_pd_4.replace({'pais':'ZAF'}, 'África do Sul', regex=True, inplace=True)
df_pd_4.replace({'pais':'GUF'}, 'Guiana Francesa', regex=True, inplace=True)
df_pd_4.replace({'pais':'MHL'}, 'Ilhas Marshall', regex=True, inplace=True)
df_pd_4.replace({'pais':'LIE'}, 'Liechtenstein', regex=True, inplace=True)
df_pd_4.replace({'pais':'BES'}, 'Países Baixos Caribenhos', regex=True, inplace=True)
df_pd_4.replace({'pais':'PRI'}, 'Porto Rico', regex=True, inplace=True)
df_pd_4.replace({'pais':'GUM'}, 'Guam', regex=True, inplace=True)
df_pd_4.replace({'pais':'ASM'}, 'Samoa Americana', regex=True, inplace=True)
df_pd_4.replace({'pais':'MNP'}, 'Comunidade das Ilhas Marianas do Norte', regex=True, inplace=True)
df_pd_4.replace({'pais':'PYF'}, 'Polinésia Francesa', regex=True, inplace=True)
df_pd_4.replace({'pais':'NCL'}, 'Nova Caledônia', regex=True, inplace=True)
df_pd_4.replace({'pais':'NIU'}, 'Niue', regex=True, inplace=True)
df_pd_4.replace({'pais':'TKL'}, 'Tokelau', regex=True, inplace=True)
df_pd_4.replace({'pais':'COK'}, 'Ilhas Cook', regex=True, inplace=True)
df_pd_4.replace({'pais':'WLF'}, 'Wallis e Futuna', regex=True, inplace=True)
df_pd_4.replace({'pais':'HTI'}, 'Haiti', regex=True, inplace=True)
df_pd_4.replace({'pais':'CAF'}, 'República Centro-Africana', regex=True, inplace=True)
df_pd_4.replace({'pais':'SSD'}, 'Sudão do Sul', regex=True, inplace=True)
df_pd_4.replace({'pais':'MWI'}, 'Malawi', regex=True, inplace=True)
df_pd_4.replace({'pais':'NGA'}, 'Nigéria', regex=True, inplace=True)
df_pd_4.replace({'pais':'TGO'}, 'Togo', regex=True, inplace=True)
df_pd_4.replace({'pais':'AGO'}, 'Angola', regex=True, inplace=True)
df_pd_4.replace({'pais':'COD'}, 'República Democrática do Congo', regex=True, inplace=True)
df_pd_4.replace({'pais':'MLI'}, 'Mali', regex=True, inplace=True)
df_pd_4.replace({'pais':'LBR'}, 'Libéria', regex=True, inplace=True)

In [ ]:
df_pd_4.head(2)

In [ ]:
# Criando um backup 
dfback = df_pd_4.copy() 

In [ ]:
# Visualizando a copia do arquivo para tratamento de incosistências 
dfback

In [ ]:
# Drop da coluna Unnamed: 0
dfback.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
dfback.head(2)

##LIMPEZA DE INCONSISTÊNCIAS ENCONTRADAS df_pd_4

In [ ]:
# Visualizando como ficou as colunas e seu tipo
dfback.dtypes

In [ ]:
# Transformei todos os valores nulos em não informados
dfback["produto"] = dfback["produto"].fillna("n_informado")
dfback["empresa"] = dfback["empresa"].fillna("n_informado")
#dfback["data_de_autorizacao"] = dfback["data_de_autorizacao"].fillna("n_informado")
#dfback["data_de_inicio"] = dfback["data_de_inicio"].fillna("n_informado")

In [ ]:
# Verificando se a alteração foi aplicada
dfback.head(3
            )

Utilizando o unique novamente, para ver as respectivas colunas e preencher os valores não informados da coluna produto e empresa

In [ ]:
# Verificando a coluna vacina
pd.unique(dfback['vacina'])
#Temos 37 resultados/ valores nulos 0

In [ ]:
# Verificando a coluna produto 
pd.unique(dfback['produto'])
#Temos 37 resultados /valores nulos 1

In [ ]:
# Verificando a coluna empresa 
pd.unique(dfback['empresa'])
#Temos 24 resultados / valores nulos 32

Realizando os filtros para fazer os devidos preenchimentos

In [ ]:
# Realizando o filtro na vacina 'Turkovac'
filtro1 = dfback.vacina == 'Turkovac'
filtro1_1 = dfback.loc[filtro1]
filtro1_1

In [ ]:
# Realizando as alterações necessárias 
filtro1_1.iat[0,2] = 'Turkovac'
filtro1_1.iat[0,3] = 'Health Institutes of Turkey'
filtro1_1

In [ ]:
# Realizando o filtro na vacina 'Julphar - Hayat-Vax'
filtro2 = dfback.vacina == 'Julphar - Hayat-Vax'
filtro2_1 = dfback.loc[filtro2]
filtro2_1

In [ ]:
# Realizando as alterações necessárias 
filtro2_1.iat[0,3] = 'Julphar'
filtro2_1.iat[1,3] = 'Julphar'
filtro2_1.iat[2,3] = 'Julphar'
filtro2_1

In [ ]:
# Realizando o filtro na vacina 'Shifa - COVIran Barakat'
filtro3 = dfback.vacina == 'Shifa - COVIran Barakat'
filtro3_1 = dfback.loc[filtro3]
filtro3_1

In [ ]:
# Realizando as alterações necessárias 
filtro3_1.iat[0,3] = 'Shifa'
filtro3_1

In [ ]:
# Realizando o filtro na vacina 'Novavax - Covavax'
filtro4 = dfback.vacina == 'Novavax - Covavax'
filtro4_1 = dfback.loc[filtro4]
filtro4_1

In [ ]:
# Realizando as alterações necessárias 
filtro4_1.iat[0,3] = 'Novavax'
filtro4_1

In [ ]:
# Realizando o filtro na vacina 'SII - Covovax'
filtro5 = dfback.vacina == 'SII - Covovax'
filtro5_1 = dfback.loc[filtro5]
filtro5_1

In [ ]:
# Realizando as alterações necessárias 
filtro5_1.iat[0,3] = 'SII'
filtro5_1.iat[1,3] = 'SII'
filtro5_1

In [ ]:
# Realizando o filtro na vacina 'Moderna – Spikevax Bivalent Original/Omicron BA.1'
filtro6 = dfback.vacina == 'Moderna – Spikevax Bivalent Original/Omicron BA.1'
filtro6_1 = dfback.loc[filtro6]
filtro6_1

In [ ]:
# Realizando as alterações necessárias 
filtro6_1.iat[0,3] = 'Moderna'
filtro6_1.iat[1,3] = 'Moderna'
filtro6_1.iat[2,3] = 'Moderna'
filtro6_1.iat[3,3] = 'Moderna'
filtro6_1.iat[4,3] = 'Moderna'
filtro6_1.iat[5,3] = 'Moderna'
filtro6_1

In [ ]:
# Realizando o filtro na vacina 'Pfizer BioNTech - Comirnaty Bivalent Original/Omicron BA.1'
filtro7 = dfback.vacina == 'Pfizer BioNTech - Comirnaty Bivalent Original/Omicron BA.1'
filtro7_1 = dfback.loc[filtro7]
filtro7_1

In [ ]:
# Realizando as alterações necessárias 
filtro7_1.iat[0,3] = 'Pfizer BioNTech'
filtro7_1.iat[1,3] = 'Pfizer BioNTech'
filtro7_1.iat[2,3] = 'Pfizer BioNTech'
filtro7_1.iat[3,3] = 'Pfizer BioNTech'
filtro7_1.iat[4,3] = 'Pfizer BioNTech'
filtro7_1.iat[5,3] = 'Pfizer BioNTech'
filtro7_1.iat[6,3] = 'Pfizer BioNTech'
filtro7_1.iat[7,3] = 'Pfizer BioNTech'
filtro7_1.iat[8,3] = 'Pfizer BioNTech'
filtro7_1

In [ ]:
# Realizando o filtro na vacina 'Pfizer BioNTech - Comirnaty Bivalent Original/Omicron BA.4/BA.5'
filtro8 = dfback.vacina == 'Pfizer BioNTech - Comirnaty Bivalent Original/Omicron BA.4/BA.5'
filtro8_1 = dfback.loc[filtro8]
filtro8_1

In [ ]:
# Realizando as alterações necessárias 
filtro8_1.iat[0,3] = 'Pfizer BioNTech'
filtro8_1.iat[1,3] = 'Pfizer BioNTech'
filtro8_1.iat[2,3] = 'Pfizer BioNTech'
filtro8_1.iat[3,3] = 'Pfizer BioNTech'
filtro8_1.iat[4,3] = 'Pfizer BioNTech'
filtro8_1.iat[5,3] = 'Pfizer BioNTech'
filtro8_1.iat[6,3] = 'Pfizer BioNTech'
filtro8_1

In [ ]:
# Realizando o filtro na vacina 'Moderna – Spikevax Bivalent Original/Omicron  - Generic'
filtro9 = dfback.vacina == 'Moderna – Spikevax Bivalent Original/Omicron  - Generic'
filtro9_1 = dfback.loc[filtro9]
filtro9_1

In [ ]:
# Realizando as alterações necessárias 
filtro9_1.iat[0,2] = 'Spikevax Bivalent Original/Omicron  - Generic'
filtro9_1.iat[0,3] = 'Moderna'
filtro9_1

In [ ]:
# Realizando o filtro na vacina 'Pfizer BioNTech - Comirnaty Bivalent Original/Omicron - Generic'
filtro10 = dfback.vacina == 'Pfizer BioNTech - Comirnaty Bivalent Original/Omicron - Generic'
filtro10_1 = dfback.loc[filtro10]
filtro10_1

In [ ]:
# Realizando as alterações necessárias 
filtro10_1.iat[0,2] = 'Comirnaty Bivalent Original/Omicron - Generic'
filtro10_1.iat[0,3] = 'Pfizer BioNTech'
filtro10_1

In [ ]:
# Realizando o filtro em todas as linhas que tem valor
filtro11 = dfback.empresa != 'n_informado'
filtro11_1 = dfback.loc[filtro11]
filtro11_1

In [ ]:
# Concatenando todos os data frames tradados em um só
df_pd4_final2 = pd.concat([filtro1_1, filtro2_1, filtro3_1, filtro4_1, filtro5_1, filtro6_1, filtro7_1, filtro8_1, filtro9_1, filtro10_1, filtro11_1], ignore_index = True)

In [ ]:
# Realizando os ultimos drop's 
df_pd4_final2.drop(columns=['vacina','fonte_de_dados'], inplace = True)

In [ ]:
# Visualizando o data frame final
df_pd4_final2.head(2)

In [ ]:
df_pd_4.groupby(['pais'],dropna=False).size().sort_values(ascending=False)

In [ ]:
df_pd_4.tail(50)

In [ ]:
df_pd_4.shape

In [ ]:
df_pd_4.isna().sum()

#INSIGHTS, FILTROS E GROUPBY df_pd_2

In [ ]:
df_pd_2_final.dtypes

In [ ]:
# Groupby para verificar a incidência de casos nos países
df_pd_2_final.groupby(['pais']).size().sort_values(ascending=False)

In [ ]:
# GroupBy para verificação de casos cumulativos por data_relatada
df_pd_2_final.groupby(['data_relatada']).size().head(10).sort_values(ascending=False).plot.bar(xlabel='casos_cumulativos', ylabel = 'Frequencia', rot=90)

#INSIGHTS, FILTROS E GROUPBY df_pd_3

In [ ]:
df_pd_3.dtypes

In [ ]:
# GroupBy com auxílio de Plot para verificação dos casos totais por regiao

In [ ]:
df_pd_3.groupby(['regiao']).size().head(10).sort_values(ascending=False).plot.bar(xlabel='casos_cumulativos_totais', ylabel = 'Frequencia', rot=90)

#INSIGHTS, FILTROS E GROUPBY df_pd_4

In [ ]:
pd.unique(df_pd_4['comeco_vacinacao'])

In [ ]:
pd.unique(df_pd_4['pais'])

In [ ]:
# GroupBy para verificação das vacinas mais utilizadas mundialmente assim como sua respectiva plotagem

df_pd_4.groupby(['produto']).size().head(10).sort_values(ascending=False).plot.bar(xlabel='nome_produto', ylabel = 'Frequencia', rot=90)

In [ ]:
# GroupBy para verificação dos períodos em que mais ocorreram a autorização das vacinas
df_pd_4.groupby(['data_de_autorizacao']).size().head(10).sort_values(ascending=False).plot.bar(xlabel='data_autorizada_vacina', ylabel = 'Frequencia', rot=90)

In [ ]:
# GroupBy para verificação dos períodos em que mais ocorreram o início da vacinação
df_pd_4.groupby(['data_de_inicio']).size().head(10).sort_values(ascending=False).plot.bar(xlabel='comeco_vacinacao', ylabel = 'Frequencia', rot=90)

In [ ]:
df_pd_4.dtypes

#PREPARAÇÃO DOS ARQUIVOS

In [ ]:
#Preparação dos arquivos tratados para salvar localmente
df_pd_2_final.to_csv('df_pd_2_global_data.csv', index=False)

df_pd_3.to_csv('df_pd_3_global_table_data.csv', index=False)

df_pd4_final2.to_csv("df_pd_4_final2_vaccination-metadata.csv", index=False)

#ENVIO DOS DATASETS PARA O CLOUD STORAGE

In [ ]:
#Enviando os Databases tratado para GCP

df_pd_2_final.to_csv('gs://projfinal/tratados/df_pd_2_global_data.csv',storage_options={'token':'token_gcp'})
df_pd_3.to_csv('gs://projfinal/tratados/df_pd_3_global_table_data.csv',storage_options={'token':'token_gcp'})
df_pd4_final2.to_csv('gs://projfinal/tratados/df_pd_4_final2_vaccination-metadata.csv',storage_options={'token':'token_gcp'})


#ENVIO PARA O MONGODB(NOSQL)

In [ ]:
# Conectando com o MongoDB

uri = "mongo_uri"
client = MongoClient(uri,tls=True,tlsCertificateKeyFile='mongo_key')


In [ ]:
# Criando coleções para enviar para o mongoDB

db = client['projfinal']
colecaotratado1 = db['tratado1']
colecaotratado2 = db['tratado2']
colecaotratado3 = db['tratado3']
colecaotratado4 = db['tratado4']

In [ ]:
# Enviando os Datasets tratados para o mongoDB

df_pd_1_final.reset_index(drop=True)
df1 = df_pd_1_final.to_dict("records")
colecaotratado1.insert_many(df1)


df_pd_2_final.reset_index(drop=True)
df2 = df_pd_2_final.to_dict("records")
colecaotratado2.insert_many(df2)

df_pd_3.reset_index(drop=True)
df3 = df_pd_3.to_dict("records")
colecaotratado3.insert_many(df3)

df_pd4_final2.reset_index(drop=True)
df4 = df_pd4_final2.to_dict("records")
colecaotratado4.insert_many(df4)

In [ ]:
# Verificação se os arquivos foram enviados corretamente para o MongoDB
colecaotratado4.count_documents({})

#FINAL DO PROCESSO